In [5]:
import numpy as np
# import tensorflow as tf
import librosa

import matplotlib.pyplot as plt

In [6]:
y, sr = librosa.load("../data/raw/maestro-v3.0.0/2018/MIDI-Unprocessed_Recital1-3_MID--AUDIO_01_R1_2018_wav--1.wav", sr=44100)

In [9]:
from dataclasses import dataclass

import note_seq
import numpy as np
# import tensorflow as tf

from typing import Iterable, List, Tuple
from note_seq import NoteSequence


@dataclass
class Note:
    pitch: int
    velocity: int


def make_frames(audio_signal: Iterable[float], sample_rate: int, frame_length: int, frame_step: int, pad_end=True, pad_value=0) -> Tuple[np.ndarray, List[float]]:
    spectrogram = librosa.feature.melspectrogram(
        y=audio_signal,
        sr=sample_rate,
        n_mels=128,
        fmin=8,
        fmax=12500,
        n_fft=1000,
        hop_length=1000 // 1,
        window="taylor"
    )
    spectrogram = librosa.power_to_db(spectrogram)
    frame_time = frame_length / sample_rate
    times = [frame_time * i for i in range(spectrogram.shape[-1])]
    return spectrogram, times


def tokenize(ns: NoteSequence, times: Iterable[float], frame_time: float, single_note=False) -> List[Note]:
    ns_sorted = sorted(ns.notes, key=lambda note: note.start_time)
    ns_iter = 0
    notes = []
    prev_notes = []

    for time in times:
        while ns_iter < len(ns_sorted) and ns_sorted[ns_iter].start_time - time < frame_time:
            prev_notes.append(ns_sorted[ns_iter])
            ns_iter += 1

        notes.append(
            [
                Note(note.pitch, note.velocity) for note in prev_notes[::-1] if note.end_time > time
            ]
        )

        if single_note and len(notes[-1]) > 1:
            notes[-1] = [notes[-1][0]]

    assert len(notes) == len(times)
    return notes


def detokenize(notes: Iterable[Iterable[Note]], times: Iterable[float], frame_time: float) -> NoteSequence:
    ns = NoteSequence()
    for notes_inner, time in zip(notes, times):
        for note in notes_inner:
            ns.notes.append(
                NoteSequence.Note(
                    pitch=note.pitch,
                    velocity=note.velocity,
                    start_time=time,
                    end_time=time + frame_time
                )
            )

    return ns


In [47]:
test_tokenize = note_seq.midi_file_to_note_sequence("../data/raw/tokenize_test.MID")

test_tokenize.notes[0].end_time = 0.3
note_seq.plot_sequence(test_tokenize)

times = [0, 0.2, 0.4, 0.6, 0.8]
notes = tokenize(test_tokenize, times, 0.2, True)

detokenized = detokenize(notes, times, 0.2)
note_seq.plot_sequence(detokenized)

Loading BokehJS ...

Loading BokehJS ...

In [48]:
test_tokenize = note_seq.midi_file_to_note_sequence("../data/raw/tokenize_test_2.MID")

note_seq.plot_sequence(test_tokenize)

c = 15
part = 2 / c
times = [part * i for i in range(c)]
notes = tokenize(test_tokenize, times, part, False)

detokenized = detokenize(notes, times, part)
note_seq.plot_sequence(detokenized)

Loading BokehJS ...

Loading BokehJS ...

In [49]:
test_tokenize = note_seq.midi_file_to_note_sequence("../data/raw/tokenize_test_3.MID")

note_seq.plot_sequence(test_tokenize)

c = 20
part = 2 / c
times = [part * i for i in range(c)]
notes = tokenize(test_tokenize, times, part, False)

detokenized = detokenize(notes, times, part)
note_seq.plot_sequence(detokenized)

Loading BokehJS ...

Loading BokehJS ...

In [50]:
test_tokenize = note_seq.midi_file_to_note_sequence("../data/raw/tokenize_test_1.MID")

note_seq.plot_sequence(test_tokenize)

times = [0.25 * i for i in range(8)]
notes = tokenize(test_tokenize, times, 0.25, False)

detokenized = detokenize(notes, times, 0.25)
note_seq.plot_sequence(detokenized)

Loading BokehJS ...

Loading BokehJS ...

In [53]:
test_notes = note_seq.midi_file_to_note_sequence("../data/raw/test_midi.MID")

note_seq.plot_sequence(test_notes)

c = 100
part = 8 / c
times = [part * i for i in range(c)]
notes = tokenize(test_notes, times, part, False)

detokenized = detokenize(notes, times, part)
note_seq.plot_sequence(detokenized)

Loading BokehJS ...

Loading BokehJS ...

In [57]:
test_notes = note_seq.midi_file_to_note_sequence("../data/raw/maestro-v3.0.0/2018/MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1.midi")

note_seq.plot_sequence(test_notes)

c = 10000
part = 600 / c
times = [part * i for i in range(c)]
notes = tokenize(test_notes, times, part, False)

detokenized = detokenize(notes, times, part)
note_seq.plot_sequence(detokenized)

Loading BokehJS ...

Loading BokehJS ...